x### Co zrobione
  ### 'Data_pierwszej_rejestracji'
    - Naprawienie formatowania 'Data_pierwszej_rejestracji'
    - Rozdzielenie 'Data_pierwszej_rejestracji' na kolumny 'Miesiac_pierwszej_rejestracji' i 'Rok_pierwszej_rejestracji'
    - Zmiana nieprawindłowych lat z 'Rok_pierwszej_rejestracji' poprzez iterowanie po indeksach roku i zamianie ich na wartości o odpowiadającym indeksie z 'Rok_produkcji'
    - Usunięcie atrybutu 'Data_pierwszej_rejestracji' (Warto się zastanowić czy zostawić usunięte, czy może zrobić merge 'Miesiac_pierwszej_rejestracji' i 'Rok_pierwszej_rejestracji')
  ### 'Generacja_pojazdu'
    - Chyba nie trzeba czyścić, braki końcowego roku zdają się być schematyczne a nie wybiórcze.  Możliwe jednak że coś przegapiłem

  ### 'Cena' i 'Waluta'
    - Wartości ceny wyrażonej w EUR zostały przekonwertowane na PLN przy pomocy API NBP wedle średniego kursu walut dnia wystawienia pojazdu lub najbliższej dostępnej dacie wśród pozostałych wartości z EUR
    - Atrybut 'Waluta' usunięty skoro  wszystkie wartości wyrażone są w PLN

  ### 'Wyposazenie'
    - Wykonany One Hot Encoding wyposażenia
    - Usunięty atrybut 'Wyposazenie'

In [1]:
from enum import unique

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df_car_sales_train_data = pd.read_csv('Data/sales_ads_train.csv',
    dtype={
        'Waluta': 'category',
        'Stan': 'category',
        'Marka_pojazdu': 'string',
        'Model_pojazdu': 'string',
        'Wersja_pojazdu': 'string',
        'Generacja_pojazdu': 'string',
        'Rok_produkcji': 'float64',
        'Przebieg_km': 'float',
        'Moc_KM': 'float',
        'Pojemnosc_cm3': 'float',
        'Rodzaj_paliwa': 'category',
        'Emisja_CO2': 'float',
        'Naped': 'category',
        'Skrzynia_biegow': 'category',
        'Typ_nadwozia': 'category',
        'Liczba_drzwi': 'float64',
        'Kolor': 'category',
        'Kraj_pochodzenia': 'category',
        'Pierwszy': 'category',
        'Lokalizacja_oferty': 'string',
        'Wyposazenie': 'string'

},
                           index_col="ID", parse_dates=['Data_pierwszej_rejestracji','Data_publikacji_oferty'], dayfirst=True)
df_car_sales_train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  int64         
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [3]:
df_car_sales_test_data = pd.read_csv('Data/sales_ads_test.csv',
    dtype={
        'Waluta': 'category',
        'Stan': 'category',
        'Marka_pojazdu': 'string',
        'Model_pojazdu': 'string',
        'Wersja_pojazdu': 'string',
        'Generacja_pojazdu': 'string',
        'Rok_produkcji': 'float64',
        'Przebieg_km': 'float',
        'Moc_KM': 'float',
        'Pojemnosc_cm3': 'float',
        'Rodzaj_paliwa': 'category',
        'Emisja_CO2': 'float',
        'Naped': 'category',
        'Skrzynia_biegow': 'category',
        'Typ_nadwozia': 'category',
        'Liczba_drzwi': 'float64',
        'Kolor': 'category',
        'Kraj_pochodzenia': 'category',
        'Pierwszy': 'category',
        'Lokalizacja_oferty': 'string',
        'Wyposazenie': 'string'

},
                           index_col="ID", parse_dates=['Data_pierwszej_rejestracji','Data_publikacji_oferty'], dayfirst=True)
df_car_sales_test_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 72907 entries, 1 to 72907
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Waluta                      72907 non-null  category      
 1   Stan                        72907 non-null  category      
 2   Marka_pojazdu               72907 non-null  string        
 3   Model_pojazdu               72907 non-null  string        
 4   Wersja_pojazdu              48394 non-null  string        
 5   Generacja_pojazdu           51734 non-null  string        
 6   Rok_produkcji               72907 non-null  float64       
 7   Przebieg_km                 72577 non-null  float64       
 8   Moc_KM                      72678 non-null  float64       
 9   Pojemnosc_cm3               72221 non-null  float64       
 10  Rodzaj_paliwa               72907 non-null  category      
 11  Emisja_CO2                  32946 non-null  float64       


# UWAGA!!
Łączenie danych testowych i treningowych tylko na czas czyszczenia stringów i formatowań!!!

In [4]:
df_car_sales_test_data['Cena'] = pd.Series()
df_car_sales = pd.concat([df_car_sales_train_data, df_car_sales_test_data], ignore_index = True)
df_car_sales = df_car_sales_train_data
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  int64         
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [5]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 24 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   Cena                        135397 non-null  int64         
 1   Waluta                      132021 non-null  category      
 2   Stan                        132075 non-null  category      
 3   Marka_pojazdu               132046 non-null  string        
 4   Model_pojazdu               132088 non-null  string        
 5   Wersja_pojazdu              87336 non-null   string        
 6   Generacja_pojazdu           93737 non-null   string        
 7   Rok_produkcji               131990 non-null  float64       
 8   Przebieg_km                 131394 non-null  float64       
 9   Moc_KM                      131664 non-null  float64       
 10  Pojemnosc_cm3               130711 non-null  float64       
 11  Rodzaj_paliwa               131987 non-null 

In [6]:
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Skrzynia_biegow,Typ_nadwozia,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie
ID,,,,,,,,,,,,,,,,,,,,,
1,13900,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,...,Manual,minivan,5.0,blue,NaN,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air..."
2,25900,PLN,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,...,Manual,station_wagon,5.0,silver,NaN,NaN,16/06/2010,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air..."
3,35900,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,...,Manual,minivan,5.0,white,Denmark,NaN,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ..."
4,5999,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,...,Manual,compact,5.0,blue,NaN,NaN,27/11/2007,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air..."
5,44800,PLN,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,...,Manual,NaN,4.0,other,Poland,Yes,20/05/2013,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499,PLN,Used,Opel,Astra,<NA>,K (2015-),2018.0,136931.0,150.0,...,Manual,compact,5.0,silver,Poland,Yes,22/08/2018,NaT,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co..."
135394,269855,PLN,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,W447 (2014-),2021.0,8.0,237.0,...,Automatic,minivan,4.0,white,NaN,NaN,NaN,2021-05-01,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa..."
135395,21900,PLN,Used,<NA>,Zafira,<NA>,B (2005-2014),NaN,179000.0,120.0,...,Manual,minivan,5.0,black,NaN,NaN,NaN,2021-05-01,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air..."


### Czyszczenie i konwersja 'Data_pierwszej_rejestracji' do typu 'datetetime64'

In [7]:
df_car_sales['Data_pierwszej_rejestracji']

ID
1                NaN
2         16/06/2010
3                NaN
4         27/11/2007
5         20/05/2013
             ...    
135393    22/08/2018
135394           NaN
135395           NaN
135396           NaN
135397    31/01/2019
Name: Data_pierwszej_rejestracji, Length: 135397, dtype: object

Zakładając brak pomyłek:

 - najstarsze auto zostało wyprodukowane w 1923, lecz dane testowe zawierają w sobie nawet takie z 1915
 - dane pochodzą z 2021

In [8]:
df_car_sales['Rok_produkcji'].describe()

count    131990.000000
mean       2012.089901
std           7.003557
min        1923.000000
25%        2008.000000
50%        2013.000000
75%        2017.000000
max        2021.000000
Name: Rok_produkcji, dtype: float64

'Data_pierwszej_rejestracji' zawiera w sobie zarówno błędne formatowanie jak i rok rejestracji

In [9]:
df_car_sales['Data_pierwszej_rejestracji'] = df_car_sales['Data_pierwszej_rejestracji'].str.replace('//', '/')

In [10]:
# Dzielenie daty rejestracji na kolumny dnia, miesiąca i roku
first_registration_expanded_data = df_car_sales['Data_pierwszej_rejestracji'].str.split('/', expand=True).rename(columns={0: 'Dzien_pierwszej_rejestracji', 1: 'Miesiac_pierwszej_rejestracji', 2: 'Rok_pierwszej_rejestracji'})

In [11]:
# Łączenie datych z kolumną roku produkci pojazdu dla dalszej analizy
df_car_sales_with_expanded_registration_dates = pd.concat([df_car_sales['Rok_produkcji'], first_registration_expanded_data], axis=1)
df_car_sales_with_expanded_registration_dates

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,
1,2005.0,NaN,NaN,NaN
2,2010.0,16,06,2010
3,2015.0,NaN,NaN,NaN
4,2007.0,27,11,2007
5,2013.0,20,05,2013
...,...,...,...,...
135393,2018.0,22,08,2018
135394,2021.0,NaN,NaN,NaN
135395,NaN,NaN,NaN,NaN


In [12]:
df_car_sales_with_expanded_registration_dates.loc[
    df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < 1940
][['Rok_produkcji', 'Rok_pierwszej_rejestracji']]

# Zastąpienie pierwszych 2 cyfr roku cyframi z roku produkcji zdaje się że rozwiąże większość konfliktów

,Rok_produkcji,Rok_pierwszej_rejestracji
ID,,
1727,2006.0,0012
1890,2016.0,0017
2265,1998.0,0098
2744,1998.0,0098
3999,2000.0,0200
...,...,...
130368,2009.0,0209
131035,2006.0,0021
132905,2006.0,0019


In [13]:
# Dodać do listy zakupów
df_car_sales.iloc[32583]

Cena                                                1300000
Waluta                                                  PLN
Stan                                                   Used
Marka_pojazdu                                   Rolls-Royce
Model_pojazdu                                       Phantom
Wersja_pojazdu                                         <NA>
Generacja_pojazdu                                      <NA>
Rok_produkcji                                        1936.0
Przebieg_km                                         40355.0
Moc_KM                                                  NaN
Pojemnosc_cm3                                        3499.0
Rodzaj_paliwa                                      Gasoline
Emisja_CO2                                              NaN
Naped                                           Rear wheels
Skrzynia_biegow                                      Manual
Typ_nadwozia                                      city_cars
Liczba_drzwi                            

In [14]:
# Usuwanie ujemnych wartości z roku
df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'] = (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].
                                                                              str.replace('-', ''))

In [15]:
# Wyświetlanie danych gdzie rok pierwszej rejestracji jest wcześniejszy niż rok produkcji
df_car_sales_with_expanded_registration_dates.loc[
    df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < df_car_sales_with_expanded_registration_dates['Rok_produkcji'].astype(float)
][['Rok_produkcji', 'Rok_pierwszej_rejestracji']]

,Rok_produkcji,Rok_pierwszej_rejestracji
ID,,
822,1998.0,1997
1089,2011.0,2010
1424,2010.0,2009
1727,2006.0,0012
1890,2016.0,0017
...,...,...
132594,2007.0,2006
132905,2006.0,0019
133581,2011.0,2010


In [16]:
# Sprawdzanie zcy Rok pierwszej rejestracji zawsze zawiera cyfry
mask = df_car_sales_with_expanded_registration_dates.loc[df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].str.contains('[\\d]') == False].index
df_car_sales_with_expanded_registration_dates.loc[mask]['Rok_pierwszej_rejestracji']


Series([], Name: Rok_pierwszej_rejestracji, dtype: object)

In [17]:
df_car_sales_with_expanded_registration_dates.loc[df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].str.contains('[0-9]{4}') == False]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,


In [18]:
test_data = df_car_sales_with_expanded_registration_dates

In [19]:
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,
822,1998.0,28,05,1997
1089,2011.0,10,10,2010
1424,2010.0,12,12,2009
1727,2006.0,20,06,0012
1890,2016.0,10,08,0017
...,...,...,...,...
132594,2007.0,28,12,2006
132905,2006.0,07,06,0019
133581,2011.0,20,06,2010


In [20]:
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,
822,1998.0,28,05,1997
1089,2011.0,10,10,2010
1424,2010.0,12,12,2009
1727,2006.0,20,06,0012
1890,2016.0,10,08,0017
...,...,...,...,...
132594,2007.0,28,12,2006
132905,2006.0,07,06,0019
133581,2011.0,20,06,2010


In [21]:
test_data.iloc[821]

Rok_produkcji                    1998.0
Dzien_pierwszej_rejestracji          28
Miesiac_pierwszej_rejestracji        05
Rok_pierwszej_rejestracji          1997
Name: 822, dtype: object

In [22]:
# Iterowanie po indeksach Rok_pierwszej_rejestracji i zastępowanie niewłaśiwyh wartości
for i in range(4):
    # Warunek
    mask = (
        df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].notnull()
        & df_car_sales_with_expanded_registration_dates['Rok_produkcji'].notnull()
        & (
            (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'])
            | (df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji'].astype(float) > 2021)
        )
    )

    # Sprawdzenie, czy są wiersze spełniające warunek
    if not df_car_sales_with_expanded_registration_dates[mask].empty:
        # Pobranie indeksów pasujących wierszy
        matching_indices = df_car_sales_with_expanded_registration_dates[mask].index

        # Iteracja przez pasujące wiersze
        for idx in matching_indices:
            # Pobranie roku produkcji i konwersja na string
            production_year = str(int(df_car_sales_with_expanded_registration_dates.loc[idx, 'Rok_produkcji']))
            # Pobranie pierwszej cyfry roku produkcji
            production_year_digit = production_year[i] if i < len(production_year) else ''

            # Pobranie roku rejestracji
            registration_year = test_data.loc[idx, 'Rok_pierwszej_rejestracji']
            # Podział roku rejestracji na głowę i ogon
            registration_year_head = registration_year[:i]
            registration_year_tail = registration_year[i + 1:]

            # Utworzenie nowej wartości roku rejestracji
            new_registration_year = registration_year_head + production_year_digit + registration_year_tail

            # Aktualizacja wartości w DataFrame
            test_data.at[idx, 'Rok_pierwszej_rejestracji'] = new_registration_year

# Sprawdzanie czy dalej występują niepoprawne wartości
test_data.loc[(test_data['Rok_pierwszej_rejestracji'].notnull() & test_data['Rok_produkcji'].notnull()) & ((test_data['Rok_pierwszej_rejestracji'].astype(float) < test_data['Rok_produkcji'].astype(float)) | (test_data['Rok_pierwszej_rejestracji'].astype(float) > 2021))]

,Rok_produkcji,Dzien_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji,Rok_pierwszej_rejestracji
ID,,,,


## Merge oczyszczony rok i miesiac pierwszej rejestracji z danymi z df_car_sales

In [23]:
df_car_sales['Rok_pierwszej_rejestracji'] = df_car_sales_with_expanded_registration_dates['Rok_pierwszej_rejestracji']
df_car_sales['Miesiac_pierwszej_rejestracji']= df_car_sales_with_expanded_registration_dates['Miesiac_pierwszej_rejestracji']
df_car_sales

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
ID,,,,,,,,,,,,,,,,,,,,,
1,13900,PLN,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,...,5.0,blue,NaN,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
2,25900,PLN,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,...,5.0,silver,NaN,NaN,16/06/2010,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010,06
3,35900,PLN,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,...,5.0,white,Denmark,NaN,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN
4,5999,PLN,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,...,5.0,blue,NaN,NaN,27/11/2007,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007,11
5,44800,PLN,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,...,4.0,other,Poland,Yes,20/05/2013,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499,PLN,Used,Opel,Astra,<NA>,K (2015-),2018.0,136931.0,150.0,...,5.0,silver,Poland,Yes,22/08/2018,NaT,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co...",2018,08
135394,269855,PLN,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,W447 (2014-),2021.0,8.0,237.0,...,4.0,white,NaN,NaN,NaN,2021-05-01,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa...",NaN,NaN
135395,21900,PLN,Used,<NA>,Zafira,<NA>,B (2005-2014),NaN,179000.0,120.0,...,5.0,black,NaN,NaN,NaN,2021-05-01,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN


# Czyszczenie 'Generacja_pojazdu'

In [24]:
car_gen = df_car_sales['Generacja_pojazdu']
car_gen

ID
1                    <NA>
2         III (2008-2016)
3           C (2011-2019)
4         Mk2 (2004-2011)
5             III (2009-)
               ...       
135393          K (2015-)
135394       W447 (2014-)
135395      B (2005-2014)
135396     II (1998-2012)
135397          E (2018-)
Name: Generacja_pojazdu, Length: 135397, dtype: string

In [25]:
# Podział 'Generacja_pojazdu' na 3 części
generation = (car_gen.str.split("(", expand=True).
              rename(columns={0:"Nazwa_generacji", 1:"Okres_produkcji"}))
# Usunięcie ')' z 'Okres_produkcji'
generation['Okres_produkcji'] = (generation['Okres_produkcji'].
                                 str.strip(")"))
# Rozdzielenie 'Okres_produkcji' na 'Poczatek_produkcji' i 'Koniec_produkcji
split_generation_years = (generation['Okres_produkcji'].
                          str.split("-", expand=True).
                          rename(columns={0: 'Poczatek_produkcji', 1:'Koniec_produkcji'})
                          )
# Usunięcie 'Okres_produkcji'
generation.drop(columns=['Okres_produkcji'], inplace=True)


#Przypisanie nowych kolumn do generation
generation['Poczatek_produkcji'] = split_generation_years['Poczatek_produkcji']
generation['Koniec_produkcji'] = split_generation_years['Koniec_produkcji']

# Pozbycie się white spaces
generation['Nazwa_generacji'] = generation['Nazwa_generacji'].str.replace("\\s", "", regex=True)
generation['Poczatek_produkcji'] = generation['Poczatek_produkcji'].str.replace("\\s", "", regex=True)
generation['Koniec_produkcji'] = generation['Koniec_produkcji'].str.replace("\\s", "", regex=True)

generation



,Nazwa_generacji,Poczatek_produkcji,Koniec_produkcji
ID,,,
1,<NA>,<NA>,<NA>
2,III,2008,2016
3,C,2011,2019
4,Mk2,2004,2011
5,III,2009,
...,...,...,...
135393,K,2015,
135394,W447,2014,
135395,B,2005,2014


In [26]:
generation["Marka_pojazdu"] = df_car_sales['Marka_pojazdu']

generation.value_counts()

Nazwa_generacji  Poczatek_produkcji  Koniec_produkcji  Marka_pojazdu
A                2008                2017              Opel             1215
E90/E91/E92/E93  2005                2012              BMW              1140
III              2013                                  Škoda            1124
B8               2014                                  Volkswagen       1074
Mk3              2010                                  Ford             1047
                                                                        ... 
II               1995                2001              Dodge               1
                                     1999              Mitsubishi          1
VII              1992                1997              Mitsubishi          1
C10              1984                1988              Mitsubishi          1
C2               1976                1982              Audi                1
Name: count, Length: 672, dtype: int64

# Konwersja EUR -> PLN

In [27]:
# Slekcja dadnych z 'Waluta' == 'EUR'
euro_to_pln = df_car_sales.loc[df_car_sales['Waluta'] == "EUR"]
euro_to_pln

,Cena,Waluta,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,...,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
ID,,,,,,,,,,,,,,,,,,,,,
736,202950,EUR,Used,Mercedes-Benz,AMG GT,<NA>,<NA>,2019.0,15.0,500.0,...,5.0,black,NaN,NaN,NaN,2021-05-01,"Słoneczna - 05555 Racibory, piaseczyński, Mazo...",[],NaN,NaN
2853,35500,EUR,Used,Rover,Other,<NA>,<NA>,1965.0,7000.0,90.0,...,4.0,beige,NaN,NaN,NaN,2021-05-03,"Maltańska 1 lok 1-070 - 61-131 Poznań, Nowe Mi...",[],NaN,NaN
2877,11000,EUR,Used,Toyota,Land Cruiser,<NA>,II (1989-1997),1992.0,51609.0,160.0,...,5.0,gray,NaN,NaN,NaN,2021-05-03,"Calle Assagador De Ricardo, 57 - 03503 Madrid,...",[],NaN,NaN
3276,80000,EUR,Used,Mercedes-Benz,Other,<NA>,<NA>,1957.0,1.0,104.0,...,2.0,silver,NaN,NaN,NaN,2021-04-26,"Oborniki, obornicki, Wielkopolskie",[],NaN,NaN
5106,4150,EUR,Used,Ford,Transit Custom,320 L2H2 PKW VA,<NA>,2017.0,183000.0,130.0,...,4.0,red,Czech Republic,Yes,NaN,2021-04-30,"Kudowa-Zdrój, kłodzki, Dolnośląskie","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130937,55500,EUR,Used,Mercedes-Benz,Other,<NA>,<NA>,1960.0,250.0,100.0,...,3.0,white,NaN,NaN,NaN,2021-05-02,"Dworcowa 30 J - 66-433 Lubiszyn, gorzowski, Lu...",[],NaN,NaN
131715,2800,EUR,Used,Ford,S-Max,2.0 TDCi DPF Titanium,I (2006-2015),2006.0,329000.0,140.0,...,5.0,other,Netherlands,NaN,05/10/2006,2021-04-25,"Lublin, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2006,10
131998,1464807,EUR,New,McLaren,720S Coupe,Standard,<NA>,2020.0,1.0,730.0,...,2.0,black,NaN,NaN,NaN,2021-04-30,"PS DAmunt 7 - 08024 Madrid, Comunidad de Madri...",[],NaN,NaN


In [28]:
import requests
import datetime
def get_exchange_rate(date: datetime, currency="EUR"):
    """
    Pobiera kurs EUR/PLN dla podanej daty.
    :param date: Data w formacie 'YYYY-MM-DD'.
    :param currency: Kod waluty (domyślnie 'EUR').
    :return: Kurs wymiany lub None, jeśli dane nie są dostępne.
    """
    url = f'https://api.nbp.pl/api/exchangerates/rates/a/{currency.lower()}/{date}/?format=json'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Sprawdza, czy odpowiedź jest poprawna
        data = response.json()
        return data["rates"][0]["mid"]  # Kurs średni
    except requests.exceptions.HTTPError as e:
        # print(f"Błąd podczas pobierania danych: {e}")
        return np.nan


In [29]:
# Selekcja unikalnych dat z 'Waluta' == 'EUR'
unique_dates = euro_to_pln['Data_publikacji_oferty'].unique()
unique_dates_df = pd.DataFrame(unique_dates, columns=["Date"])
unique_dates_df

,Date
0,2021-05-01
1,2021-05-03
2,2021-04-26
3,2021-04-30
4,2021-05-04
5,2021-04-28
6,2021-04-27
7,2021-05-02
8,2021-04-29
9,2021-04-24


In [30]:
# Tworzenie słownika z wartościami wymiany
dict_values = {}
list_of_exchange_rates = []
for i, index in enumerate(unique_dates.tolist()):
    exchange_rate = get_exchange_rate(str(index).split(" ")[0], 'EUR')
    dict_values[index] = (float(exchange_rate))
    list_of_exchange_rates.append(exchange_rate)
unique_dates_df['Exchange_rate'] = pd.Series(list_of_exchange_rates).bfill().ffill()
unique_dates_df

,Date,Exchange_rate
0,2021-05-01,4.5565
1,2021-05-03,4.5565
2,2021-04-26,4.5565
3,2021-04-30,4.5654
4,2021-05-04,4.5539
5,2021-04-28,4.5811
6,2021-04-27,4.5613
7,2021-05-02,4.5782
8,2021-04-29,4.5782
9,2021-04-24,4.5562


In [31]:

for row in unique_dates_df.itertuples():
    mask = euro_to_pln.loc[euro_to_pln['Data_publikacji_oferty'] == row[1]].index
    mask_len = len(mask)
    exchange_rate_numpy = pd.Series(np.full(mask_len, row[2]))
    df_car_sales.loc[mask, 'Cena'] = df_car_sales.loc[mask, 'Cena'] * row[2]
df_car_sales.drop(columns=['Waluta'], inplace=True)
df_car_sales

C:\Users\vipcy\AppData\Local\Temp\ipykernel_9800\3606182654.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[924741.675   31439.85    11846.9     42311.659   51643.371  118469.
 196157.325   33713.5435  42311.659 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_car_sales.loc[mask, 'Cena'] = df_car_sales.loc[mask, 'Cena'] * row[2]


,Cena,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Liczba_drzwi,Kolor,Kraj_pochodzenia,Pierwszy_wlasciciel,Data_pierwszej_rejestracji,Data_publikacji_oferty,Lokalizacja_oferty,Wyposazenie,Rok_pierwszej_rejestracji,Miesiac_pierwszej_rejestracji
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,1998.0,...,5.0,blue,NaN,NaN,NaN,2021-04-28,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN
2,25900.0,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,1598.0,...,5.0,silver,NaN,NaN,16/06/2010,2021-05-04,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)","['ABS', 'Electric front windows', 'Drivers air...",2010,06
3,35900.0,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,1598.0,...,5.0,white,Denmark,NaN,NaN,2021-05-03,"Sianów, koszaliński, Zachodniopomorskie","['ABS', 'Electric front windows', 'Passengers ...",NaN,NaN
4,5999.0,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,1560.0,...,5.0,blue,NaN,NaN,27/11/2007,2021-05-02,"Gdańsk, Pomorskie, Przymorze Wielkie","['ABS', 'Electric front windows', 'Drivers air...",2007,11
5,44800.0,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,1798.0,...,4.0,other,Poland,Yes,20/05/2013,2021-05-02,"Świdnik, świdnicki, Lubelskie","['ABS', 'Electric front windows', 'Drivers air...",2013,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,Used,Opel,Astra,<NA>,K (2015-),2018.0,136931.0,150.0,1598.0,...,5.0,silver,Poland,Yes,22/08/2018,NaT,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)","['ABS', 'ASR (traction control)', 'On-board co...",2018,08
135394,269855.0,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,W447 (2014-),2021.0,8.0,237.0,1950.0,...,4.0,white,NaN,NaN,NaN,2021-05-01,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)","['ABS', 'Electrically adjustable mirrors', 'Pa...",NaN,NaN
135395,21900.0,Used,<NA>,Zafira,<NA>,B (2005-2014),NaN,179000.0,120.0,1700.0,...,5.0,black,NaN,NaN,NaN,2021-05-01,"WALERIANY 30 A - 96-330 Waleriany, żyrardowski...","['ABS', 'Electric front windows', 'Drivers air...",NaN,NaN


# 'Wyposazenie'

In [32]:
# Wylistowanie wszystkich elementów z atrybutu 'Wyposazenie'
equipment_list = df_car_sales['Wyposazenie'].dropna().str.strip("[]").str.replace("'", "").str.split(", ")
# Zredukowanie listy do set'a z unikalnymi wartościami
unique_equipment = set([item for sublist in equipment_list for item in sublist if item])

# Stworzenie One Hot encoded atrybutu dla każdego elementu
for item in unique_equipment:
    df_car_sales[item] = df_car_sales['Wyposazenie'].str.contains(item, regex=False)
df_car_sales.drop(columns=['Wyposazenie'], inplace=True)
df_car_sales

,Cena,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Twilight sensor,Electrically adjustable seats,Rain sensor,USB socket,Central locking,Heated front seats,HUD(head-up display),Daytime running lights,Shift paddles,Dual zone air conditioning
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,1998.0,...,False,False,True,True,True,False,False,False,False,True
2,25900.0,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,1598.0,...,True,False,True,False,True,True,False,False,False,False
3,35900.0,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,1598.0,...,False,False,True,True,True,True,False,False,False,False
4,5999.0,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,1560.0,...,False,False,False,False,True,False,False,False,False,False
5,44800.0,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,1798.0,...,False,False,False,False,True,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,Used,Opel,Astra,<NA>,K (2015-),2018.0,136931.0,150.0,1598.0,...,False,False,False,False,False,False,False,False,False,False
135394,269855.0,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,W447 (2014-),2021.0,8.0,237.0,1950.0,...,True,False,True,True,True,True,False,False,True,False
135395,21900.0,Used,<NA>,Zafira,<NA>,B (2005-2014),NaN,179000.0,120.0,1700.0,...,True,False,True,True,True,False,False,True,False,False


In [33]:
df_car_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 135397 entries, 1 to 135397
Data columns (total 94 columns):
 #   Column                           Non-Null Count   Dtype         
---  ------                           --------------   -----         
 0   Cena                             135397 non-null  float64       
 1   Stan                             132075 non-null  category      
 2   Marka_pojazdu                    132046 non-null  string        
 3   Model_pojazdu                    132088 non-null  string        
 4   Wersja_pojazdu                   87336 non-null   string        
 5   Generacja_pojazdu                93737 non-null   string        
 6   Rok_produkcji                    131990 non-null  float64       
 7   Przebieg_km                      131394 non-null  float64       
 8   Moc_KM                           131664 non-null  float64       
 9   Pojemnosc_cm3                    130711 non-null  float64       
 10  Rodzaj_paliwa                    131987 non-null 

# Lokalizacja_oferty

In [34]:
# df_car_sales.loc[ df_car_sales['Lokalizacja_oferty'].str.contains('[\\d]{2}-[\\d]{3}', regex=True) == False]['Lokalizacja_oferty']
# df_car_sales.loc[df_car_sales['Lokalizacja_oferty'].str.contains('([\\d]{2}\\w*-?\\w*[\\d]{3}\\s\\w*\\s*),', regex=True)]['Lokalizacja_oferty']
# df_car_sales.loc[ df_car_sales['Lokalizacja_oferty'].str.contains('[(Polska]', regex=True) == False &  df_car_sales['Lokalizacja_oferty'].str.contains('[(]', regex=True) ]['Lokalizacja_oferty']
df = df_car_sales.loc[df_car_sales['Lokalizacja_oferty'].str.contains("\\([\\w*]")]['Lokalizacja_oferty']
df.loc[df.str.contains("Polska") == False]

ID
2860      grensstraat - 2243 Antwerpia, antwerpia, Belgi...
2877      Calle Assagador De Ricardo, 57 - 03503 Madrid,...
7505      OC Centro, Sjízdná 4, OSTRAVA - 722 00 Novy Ji...
9868      grensstraat - 2243 Antwerpia, antwerpia, Belgi...
10015     Calle Assagador De Ricardo, 57 - 03503 Madrid,...
                                ...                        
122378    Calle Assagador De Ricardo, 57 - 03503 Madrid,...
126627    K mostu 393 Stara ves nad Ondřejnici - 73923 N...
129702    K mostu 393 Stara ves nad Ondřejnici - 73923 N...
131998    PS DAmunt 7 - 08024 Madrid, Comunidad de Madri...
133348    OC Centro, Sjízdná 4, OSTRAVA - 722 00 Novy Ji...
Name: Lokalizacja_oferty, Length: 65, dtype: string

In [35]:
localization_df = pd.DataFrame()
localization_df['Lokalizacja_oferty'] = df_car_sales['Lokalizacja_oferty']
localization_df["Kod_pocztowy"] = df_car_sales["Lokalizacja_oferty"].str.extract('([\\d]{2}\\w*-?\\w*[\\d]{3}s*)')
localization_df["Miejscowosc"] = df_car_sales["Lokalizacja_oferty"].str.extract('[\\d]{2}\\w*-?\\w*[\\d]{3}\\s(\\w*\\s*\\w*)')
localization_df.loc[localization_df["Miejscowosc"].isnull()] = localization_df.loc[localization_df["Miejscowosc"].isnull()]["Miejscowosc"].str.extract('^(\\w*),')
localization_df["Ulica"] = df_car_sales["Lokalizacja_oferty"].str.extract('(ul\\.?\\s*?[A-Za-ząćęłńóśźżĄĆĘŁŃÓŚŹŻ\\s]*)')
localization_df.loc[localization_df["Lokalizacja_oferty"].notnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica
ID,,,,
1,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...",99-300,Kutno,<NA>
2,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)",41-407,Imielin,ul. Wiosenna
7,Ul. Buforowa 125 / Hala nr 23A - 52-131 Wrocła...,52-131,Wrocław,<NA>
11,"ul. Radzymińska 78 - 03-752 Warszawa, Praga-Pó...",03-752,Warszawa,ul. Radzymińska
12,"Zatorze 1 - 02-852 Warszawa, Mazowieckie (Polska)",02-852,Warszawa,<NA>
...,...,...,...,...
135392,"Kasprzaka 1 - 66-400 Gorzów Wielkopolski, Lubu...",66-400,Gorzów Wielkopolski,<NA>
135393,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)",53-012,Wrocław,<NA>
135394,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)",26-600,Radom,<NA>


In [36]:
cities_df = pd.read_csv('woj/wojewodztwa_miasta.csv', sep=';', index_col=0)
unique_wojewodztwa = cities_df["nazwa województwa"].unique()
cities_df

,nazwa miasta,nazwa województwa
id,,
1,Bolesławiec,DOLNOŚLĄSKIE
2,Nowogrodziec,DOLNOŚLĄSKIE
3,Bielawa,DOLNOŚLĄSKIE
4,Dzierżoniów,DOLNOŚLĄSKIE
5,Pieszyce,DOLNOŚLĄSKIE
...,...,...
914,Resko,ZACHODNIOPOMORSKIE
915,Węgorzyno,ZACHODNIOPOMORSKIE
916,Koszalin,ZACHODNIOPOMORSKIE


In [37]:
localization_df["Wojewodztwo"] = pd.Series(dtype='str')
wojewodztwa = cities_df.groupby(by='nazwa województwa').count().index.tolist()
for index in wojewodztwa:
    # print(index)
    mask = localization_df.loc[localization_df["Lokalizacja_oferty"].str.upper().str.contains(index) & localization_df["Lokalizacja_oferty"].notnull()].index
    localization_df.loc[mask, 'Wojewodztwo'] = index
    # print(localization_df.iloc[mask])

    # localization_df[mask, ["Wojewodztwo"]] = index
localization_df.loc[localization_df["Wojewodztwo"].notnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo
ID,,,,,
1,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...",99-300,Kutno,<NA>,ŁÓDZKIE
12,"Zatorze 1 - 02-852 Warszawa, Mazowieckie (Polska)",02-852,Warszawa,<NA>,MAZOWIECKIE
14,"Harcerska, 64 - 43-100 Tychy, Śląskie (Polska)",43-100,Tychy,<NA>,ŚLĄSKIE
18,"Obornicka 72-74 - 62-002 Suchy Las, poznański,...",62-002,Suchy Las,<NA>,WIELKOPOLSKIE
19,"ul. Stalowa 16 - 41-506 Chorzów, Śląskie (Polska)",41-506,Chorzów,ul. Stalowa,ŚLĄSKIE
...,...,...,...,...,...
135386,"ul. Stalowa 16 - 41-506 Chorzów, Śląskie (Polska)",41-506,Chorzów,ul. Stalowa,ŚLĄSKIE
135392,"Kasprzaka 1 - 66-400 Gorzów Wielkopolski, Lubu...",66-400,Gorzów Wielkopolski,<NA>,LUBUSKIE
135394,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)",26-600,Radom,<NA>,MAZOWIECKIE


In [38]:
localization_df["Miejscowosc2"] = pd.Series(dtype='str')
cities = cities_df.groupby(by='nazwa miasta').count().index.tolist() + ["Warszawa"]
for index in cities:
    # print(index)
    mask = localization_df.loc[localization_df["Lokalizacja_oferty"].str.upper().str.contains(index.upper()) & localization_df["Lokalizacja_oferty"].notnull()].index
    localization_df.loc[mask, 'Miejscowosc2'] = index
    # print(localization_df.iloc[mask])

    # localization_df[mask, ["Wojewodztwo"]] = index
localization_df.loc[localization_df["Miejscowosc2"].isnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo,Miejscowosc2
ID,,,,,,
3,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,<NA>,<NA>,<NA>,<NA>,NaN,NaN
5,<NA>,<NA>,<NA>,<NA>,NaN,NaN
6,<NA>,<NA>,<NA>,<NA>,NaN,NaN
8,<NA>,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...
135385,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135388,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135389,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [39]:
localization_df.loc[localization_df["Lokalizacja_oferty"].isnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo,Miejscowosc2
ID,,,,,,
3,<NA>,<NA>,<NA>,<NA>,NaN,NaN
4,<NA>,<NA>,<NA>,<NA>,NaN,NaN
5,<NA>,<NA>,<NA>,<NA>,NaN,NaN
6,<NA>,<NA>,<NA>,<NA>,NaN,NaN
8,<NA>,<NA>,<NA>,<NA>,NaN,NaN
...,...,...,...,...,...,...
135384,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135385,<NA>,<NA>,<NA>,<NA>,NaN,NaN
135388,<NA>,<NA>,<NA>,<NA>,NaN,NaN


In [40]:
mask = localization_df.loc[localization_df["Miejscowosc"].notnull()].index
localization_df.loc[mask, "Miejscowosc2"] = localization_df.loc[mask, "Miejscowosc"]
localization_df.loc[localization_df["Miejscowosc2"].notnull()]


,Lokalizacja_oferty,Kod_pocztowy,Miejscowosc,Ulica,Wojewodztwo,Miejscowosc2
ID,,,,,,
1,"SŁONECZNA 1 - 99-300 Kutno, kutnowski, Łódzkie...",99-300,Kutno,<NA>,ŁÓDZKIE,Kutno
2,"ul. Wiosenna 8 - 41-407 Imielin, Centrum (Polska)",41-407,Imielin,ul. Wiosenna,NaN,Imielin
7,Ul. Buforowa 125 / Hala nr 23A - 52-131 Wrocła...,52-131,Wrocław,<NA>,NaN,Wrocław
11,"ul. Radzymińska 78 - 03-752 Warszawa, Praga-Pó...",03-752,Warszawa,ul. Radzymińska,NaN,Warszawa
12,"Zatorze 1 - 02-852 Warszawa, Mazowieckie (Polska)",02-852,Warszawa,<NA>,MAZOWIECKIE,Warszawa
...,...,...,...,...,...,...
135392,"Kasprzaka 1 - 66-400 Gorzów Wielkopolski, Lubu...",66-400,Gorzów Wielkopolski,<NA>,LUBUSKIE,Gorzów Wielkopolski
135393,"Wyścigowa 58A - 53-012 Wrocław, Krzyki (Polska)",53-012,Wrocław,<NA>,NaN,Wrocław
135394,"Garbarska 79 - 26-600 Radom, Mazowieckie (Polska)",26-600,Radom,<NA>,MAZOWIECKIE,Radom


In [41]:
df_car_sales["Wojewodztwo"] = localization_df["Wojewodztwo"]
df_car_sales["Miejscowosc"] = localization_df["Miejscowosc2"]
# df_car_sales[["Lokalizacja_oferty", "Wojewodztwo", "Miejscowosc"]]

In [43]:
df_car_sales.drop(columns="Lokalizacja_oferty", inplace=True)

In [46]:
df_car_sales

,Cena,Stan,Marka_pojazdu,Model_pojazdu,Wersja_pojazdu,Generacja_pojazdu,Rok_produkcji,Przebieg_km,Moc_KM,Pojemnosc_cm3,...,Rain sensor,USB socket,Central locking,Heated front seats,HUD(head-up display),Daytime running lights,Shift paddles,Dual zone air conditioning,Wojewodztwo,Miejscowosc
ID,,,,,,,,,,,,,,,,,,,,,
1,13900.0,Used,Renault,Grand Espace,Gr 2.0T 16V Expression,<NA>,2005.0,213000.0,170.0,1998.0,...,True,True,True,False,False,False,False,True,ŁÓDZKIE,Kutno
2,25900.0,Used,Renault,Megane,1.6 16V 110,III (2008-2016),2010.0,117089.0,110.0,1598.0,...,True,False,True,True,False,False,False,False,NaN,Imielin
3,35900.0,Used,Opel,Zafira,Tourer 1.6 CDTI ecoFLEX Start/Stop,C (2011-2019),2015.0,115600.0,136.0,1598.0,...,True,True,True,True,False,False,False,False,NaN,NaN
4,5999.0,Used,Ford,Focus,1.6 TDCi FX Silver / Silver X,Mk2 (2004-2011),2007.0,218000.0,90.0,1560.0,...,False,False,True,False,False,False,False,False,NaN,NaN
5,44800.0,Used,Toyota,Avensis,1.8,III (2009-),2013.0,NaN,NaN,1798.0,...,False,False,True,False,False,True,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135393,45499.0,Used,Opel,Astra,<NA>,K (2015-),2018.0,136931.0,150.0,1598.0,...,False,False,False,False,False,False,False,False,NaN,Wrocław
135394,269855.0,New,Mercedes-Benz,Vito,124 CDI Tourer Lang,W447 (2014-),2021.0,8.0,237.0,1950.0,...,True,True,True,True,False,False,True,False,MAZOWIECKIE,Radom
135395,21900.0,Used,<NA>,Zafira,<NA>,B (2005-2014),NaN,179000.0,120.0,1700.0,...,True,True,True,False,False,True,False,False,MAZOWIECKIE,Waleriany


KeyError: 'Lokalizacja_oferty'

Note:
- zastnanowić się czy wartości numerycznych nie zamienić na przedziały
- Uwzględnić nieproporcjonalność danych, rozważyć zastosowanie Oversamplingu


- Kiedy NIE usuwać wierszy?

    Gdy braki są systematyczne (np. wszystkie samochody marki "Opel" mają brakującą Emisja_CO2) – usuwanie wprowadzi bias.

    Gdy dane są rzadkie (np. tylko 100 wierszy), a usunięcie wierszy znacząco zmniejszy próbkę.

    Gdy można uzupełnić braki (np. średni Przebieg_km dla danej marki/roku).
